In [1]:
import import_ipynb

In [2]:
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from models import base_model
from utils import load_model, SNR, sum_loss, compile_model
import h5py


Using TensorFlow backend.


importing Jupyter notebook from models.ipynb
importing Jupyter notebook from utils.ipynb


In [3]:
def load_h5_list(dirname):
    datasets = []
    filenames = os.listdir(dirname)
    file_extensions = set(['.h5'])
    for filename in filenames:
        ext = os.path.splitext(filename)[-1]
        if ext in file_extensions:
            full_filename = os.path.join(dirname, filename)
            datasets.append(full_filename)
    return datasets

In [4]:
def load_h5(dataset_name):
    print('Loading dataset ',dataset_name)
    with h5py.File(dataset_name, 'r') as hf:
        X = (hf['data'][:])
        Y = (hf['label'][:])
    print(X.shape)
    print(Y.shape)
    return X, Y

In [7]:
def run():
    # Hyper params
    EPOCHS = 1
    BATCH_SIZE = 256
    LOAD_WEIGHTS = True
    WEIGHTS_PATH = 'weights/'
    WEIGHTS_FILE = 'asr-weights.hdf5'
    VALID_SPLIT = 0.05
    SHUFFLE = True
    MINI_EPOCH = 3 # set each dataset's epochs
    
    model = base_model()
    model = load_model(model, os.path.join(WEIGHTS_PATH, WEIGHTS_FILE), load_weights=LOAD_WEIGHTS)
    model = compile_model(model)
    
    datasets = load_h5_list('data/')
    checkpointer = ModelCheckpoint(filepath=os.path.join(WEIGHTS_PATH, WEIGHTS_FILE), verbose=1, save_best_only=True) 
    earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
    
    for i in range(EPOCHS):
        print('#REAL EPOCH(%3d/%3d)'%(i+1,EPOCHS))
        for dataset in datasets:
            X,Y = load_h5(dataset)
            model.fit(X, Y,
                    batch_size=BATCH_SIZE,
                    epochs=MINI_EPOCH, 
                    shuffle=SHUFFLE,
                    callbacks=[checkpointer, earlystopper],
                    validation_split=VALID_SPLIT)
    print('Training Finished')

In [8]:
if __name__ == '__main__':
    run()
    #print('Model already trained!')

load base model..
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 1)       0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 128, 16)      272         input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_8 (LeakyReLU)       (None, 128, 16)      0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 64, 32)       4128        leaky_re_lu_8[0][0]              
__________________________________________________________________________

load model weights success!
model compilation success!
#REAL EPOCH(  1/  1)
Loading dataset  data/asr-ex10-start0-end9-scale8-sr48000-dim256-strd64-train.h5
(22031, 256, 1)
(22031, 256, 1)
Train on 20929 samples, validate on 1102 samples
Epoch 1/3
20929/20929 [==============================] - 11s 533us/step - loss: 6.2195e-05 - sum_loss: 4.0638 - SNR: 15.3516 - val_loss: 1.6534e-04 - val_sum_loss: 9.3287 - val_SNR: 10.0173

Epoch 00001: val_loss improved from inf to 0.00017, saving model to weights/asr-weights.hdf5
Epoch 2/3
20929/20929 [==============================] - 11s 508us/step - loss: 6.1496e-05 - sum_loss: 4.0181 - SNR: 15.3729 - val_loss: 1.6396e-04 - val_sum_loss: 9.2509 - val_SNR: 10.0104

Epoch 00002: val_loss improved from 0.00017 to 0.00016, saving model to weights/asr-weights.hdf5
Epoch 3/3
20929/20929 [==============================] - 11s 504us/step - loss: 6.0684e-05 - sum_loss: 3.9650 - SNR: 15.4158 - val_loss: 1.2796e-04 - val_sum_loss: 7.2198 - val_SNR: 10.3305
